In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

import sys
sys.path.append("../../")

from graph import DataExtractor

/tmp/ipykernel_307509/656216145.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from graph import DataExtractor


In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [9]:
voltai_graph = DataExtractor(
    folder_path="/home/aiffel04/AIFFEL/VOLTAI/data/papers/", 
    file_number=140
).graph

In [10]:
# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

In [11]:
# 4개의 질문 입력
questions = [
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "CAM (Cathode Active Material)": {
                "Stoichiometry information": "NCM-622",
                "Commercial NCM": "No",
                "Lithium source": "LiOH",
                "Synthesis method": "co-precipitation",
                "Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.": "Hydrothermal",
                "What is the crystallization temperature used in the process? Specify all related temperatures (e.g., calcination or sintering) mentioned for the crystallization stage.": "100°C",
                "What is the time duration for the crystallization process, including any calcination or sintering stages? Specify the hours or minutes mentioned in the experimental procedure.": "12 hr",
                "Doping": "Zr4+ doping",
                "Coating": "ZrO2 coating",
                "Additional treatment": "None"
                }
        }
    ]
    """,    
    
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "Electrode (only for coin-cell (half-cell))": {
                "Active material : Conductive additive : Binder ratio": "90 : 5 : 5",
                "Electrolyte": "LiPF6 (EC, EMC, DEC mixture in a 1:1:1 volume ratio)",
                "Additive": "FEC 10% addition",
                "Electrode thickness": "100 µm",
                "Only Cathode Electrode diameter": "14π",
                "Loading density (mass loading of NCM)": "0.005 g/cm^2",
                "Additional treatment for electrode": "None"
                },
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {        
            "Morphological results": {
                "Explanation of SEM results": "Fig. 2a, b; the NCM-622 seems to have more or less a spherical morphology with a diameter of 3–5 µm, composed of densely packed primary particles",
                "Explanation of TEM results": "None"
                },    
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {            
            "Cathode Performance": {
                "Capacity at all C-rate, mAh/g (with electrode state)": [{
                "214.5 mAh/g": "@0.1C, ZrO2-coated",
                "200.8 mAh/g": "@0.5C, ZrO2-coated"
                }],
                "Voltage range": "2.8–4.3 V",
                "Temperature": "Room temperature and 55°C"
                }
        }
    ]
    """
]


In [12]:
# 4개의 질문에 대해 그래프 실행 및 출력
outputs = []
for i, question in enumerate(questions):
    # inputs = GraphState(question=question)
    print(f"#####   {i}번째 질문   #####")
    # print(question)
    result = voltai_graph.invoke(
        input={"question":question},
        config=config,
    )
    outputs.append(result)

#####   0번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   1번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   2번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   3번째 질문   #####
==== [RELEVANCE CHECK] ====
yes


In [13]:
import pprint
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
pprint.pprint(outputs_dict)

{'Active material : Conductive additive : Binder ratio': '80 : 10 : 10',
 'Additional treatment': 'None',
 'Additional treatment for electrode': 'None',
 'Additive': 'None',
 'Capacity at all C-rate, mAh/g (with electrode state)': [{'207 mAh/g': '@0.2C, '
                                                                        'concentration-gradient '
                                                                        'Li[Ni0.67Co0.15Mn0.18]O2',
                                                           '216 mAh/g': '@0.2C, '
                                                                        'core '
                                                                        'Li[Ni0.8Co0.15Mn0.05]O2'}],
 'Coating': 'None',
 'Commercial NCM': 'No',
 'Crystallization method': 'None',
 'Crystallization temperature': 'None',
 'Crystallization time': 'None',
 'Doping': 'None',
 'Electrode thickness': 'None',
 'Electrolyte': '1 M LiPF6 in a mixture of ethylene carbonate and diethyl '
   

In [14]:
# 알파벳순 말고 그대로 출력하는 코드
import json

print(json.dumps(outputs_dict, indent=4, ensure_ascii=False))


{
    "Stoichiometry information": "Li[Ni0.67Co0.15Mn0.18]O2",
    "Commercial NCM": "No",
    "Lithium source": "LiOH",
    "Synthesis method": "co-precipitation",
    "Crystallization method": "None",
    "Crystallization temperature": "None",
    "Crystallization time": "None",
    "Doping": "None",
    "Coating": "None",
    "Additional treatment": "None",
    "Active material : Conductive additive : Binder ratio": "80 : 10 : 10",
    "Electrolyte": "1 M LiPF6 in a mixture of ethylene carbonate and diethyl carbonate, 1:1 in volume",
    "Additive": "None",
    "Electrode thickness": "None",
    "Only Cathode Electrode diameter": "None",
    "Loading density (mass loading of NCM)": "None",
    "Additional treatment for electrode": "None",
    "Explanation of SEM results": "Fig. 1 shows SEM images of core and concentration-gradient precursors and lithiated oxides. The particles are spherical, and the diameter of core and concentration-gradient particles was estimated to be about 10 a

In [34]:
pd.DataFrame(outputs_dict)

,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,Crystallization method,Crystallization temperature,Crystallization time,Doping,Coating,Additional treatment,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,"Capacity at all C-rate, mAh/g (with electrode state)",Voltage range,Temperature
0,NCM-622,No,None,simple wet-chemical coating,None,None,None,Zr4+ doping,ZrO2 coating,None,...,None,None,None,0.004-0.005 g/cm^2,Dried at 80 °C overnight in the vacuum oven,"Fig. 2a, b; the NCM-622 seems to have more or ...","HR-TEM images zirconia coating in Fig. 2c, d c...","{'214.65 mAh/g': '@0.1C, ZrO2-coated', 'None':...",2.8–4.3 V,Room temperature and 55°C
